In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow
import os

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [2]:
TRACK_DIR = os.path.abspath('.')
INTRO_DIR = os.path.join(os.path.dirname(TRACK_DIR), '01-intro')
DATA_DIR = os.path.join(INTRO_DIR, 'data')

In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe(os.path.join(DATA_DIR, 'green_tripdata_2021-01.parquet'))
df_val = read_dataframe(os.path.join(DATA_DIR, 'green_tripdata_2021-02.parquet'))

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Model Management

In [37]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
logged_model = 'runs:/0810809d13434e87b3a072ceadf76170/models_mlflow'

# Load model as a PyFuncModel.
xgb_model = mlflow.xgboost.load_model(logged_model)

[20:40:32] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [39]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

y_pred = xgb_model.predict(valid)

In [40]:
y_pred[:10]

array([15.5802555,  7.1420774, 15.14117  , 24.267075 ,  9.163609 ,
       17.033915 , 12.7239485,  8.841861 ,  8.936092 , 20.295572 ],
      dtype=float32)

## MLflow's Model Registry

In [6]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='./mlruns/1', creation_time=1675888710859, experiment_id='1', last_update_time=1675888710859, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1675888418303, experiment_id='0', last_update_time=1675888418303, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [17]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [18]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ff73ec842caa46f5b5f18f63bcf30ca1, rmse: 6.3023
run id: 99ab671f7a84406daa292ed2b318d9ec, rmse: 6.3023
run id: b33ded2fc8424eb18e0db789cbbefff2, rmse: 6.3023
run id: 6715fba0d61d456dbb17cd6935f3793b, rmse: 6.3107
run id: c9b6a585bfd34f9eb52040342e642a75, rmse: 6.3402


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [19]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [34]:
run_id = "99ab671f7a84406daa292ed2b318d9ec"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/02/16 15:50:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1676573426692, current_stage='None', description=None, last_updated_timestamp=1676573426692, name='nyc-taxi-regressor', run_id='99ab671f7a84406daa292ed2b318d9ec', run_link=None, source='./mlruns/1/99ab671f7a84406daa292ed2b318d9ec/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [35]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


In [36]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1676573426692, current_stage='Staging', description=None, last_updated_timestamp=1676573437046, name='nyc-taxi-regressor', run_id='99ab671f7a84406daa292ed2b318d9ec', run_link=None, source='./mlruns/1/99ab671f7a84406daa292ed2b318d9ec/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [37]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1676573426692, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2023-02-16', last_updated_timestamp=1676573441495, name='nyc-taxi-regressor', run_id='99ab671f7a84406daa292ed2b318d9ec', run_link=None, source='./mlruns/1/99ab671f7a84406daa292ed2b318d9ec/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [25]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [26]:
df = read_dataframe(os.path.join(DATA_DIR, 'green_tripdata_2021-03.parquet'))

In [28]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_21062/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/wa/PROJETOS/MLEngineer-Studies/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [29]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [31]:
X_test = preprocess(df, dv)

In [32]:
target = "duration"
y_test = df[target].values

In [38]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[15:51:25] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 48.9 s, sys: 293 ms, total: 49.2 s
Wall time: 18.4 s


{'rmse': 6.24904641701084}

In [39]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[15:55:24] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 1min, sys: 440 ms, total: 1min
Wall time: 25.8 s


{'rmse': 6.24904641701084}

In [53]:
6.24904641701084 > 6.24904641701084

False

In [48]:
model_production = client.get_latest_versions(name=model_name, stages=['production'])[0]
model_production

<ModelVersion: creation_timestamp=1676573426692, current_stage='Production', description='The model version 2 was transitioned to Staging on 2023-02-16', last_updated_timestamp=1676573462954, name='nyc-taxi-regressor', run_id='99ab671f7a84406daa292ed2b318d9ec', run_link=None, source='./mlruns/1/99ab671f7a84406daa292ed2b318d9ec/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [49]:
model_production.source

'./mlruns/1/99ab671f7a84406daa292ed2b318d9ec/artifacts/model'

In [50]:
model = mlflow.pyfunc.load_model(model_production.source)

[16:01:34] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [52]:
y_pred = model.predict(X_test)
{"rmse": mean_squared_error(y_test, y_pred, squared=False)}

{'rmse': 6.24904641701084}

In [54]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1676572244176, current_stage='Production', description='The model version 1 was transitioned to Staging on 2023-02-16', last_updated_timestamp=1676574335285, name='nyc-taxi-regressor', run_id='ff73ec842caa46f5b5f18f63bcf30ca1', run_link=None, source='./mlruns/1/ff73ec842caa46f5b5f18f63bcf30ca1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>